In [ ]:
import httpx
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
key = os.environ['AZURE_OPENAI_API_KEY']
endpoint = os.environ['AZURE_OPENAI_ENDPOINT']
version = os.environ['OPENAI_API_VERSION']
model = 'gpt-4o'

In [ ]:
# Set up the variables
url = f"{endpoint}/openai/deployments/tts/audio/speech?api-version=2024-02-15-preview"
headers = {
    "api-key": key,
    "Content-Type": "application/json"
}
data = {
    "model": "tts-1-hd",
    "input": "I'm excited to try text to speech.",
    "voice": "alloy"
}

# Make the POST request
response = httpx.post(url, headers=headers, json=data)

# Save the output to a file
with open("speech.mp3", "wb") as f:
    f.write(response.content)

In [40]:
from libs.community.langchain_community.tools.azure_openai.text2speech import AzureOpenAIText2SpeechTool
from langchain_openai import AzureChatOpenAI

In [7]:
model = AzureChatOpenAI(
    openai_api_version=version,  
    azure_deployment=model,
    azure_endpoint=endpoint,
    temperature=0.5,
    max_tokens=200,
    timeout=60,
    max_retries=10,
    # organization="...",
    # model="gpt-35-turbo",
    # model_version="0125",
    # other params...
    )

In [41]:
tts = AzureOpenAIText2SpeechTool(azure_openai_key=key, openai_endpoint=endpoint)

In [42]:
model.bind_tools([tts])

RunnableBinding(bound=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000025AA2859670>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000025AA285AAE0>, temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='', request_timeout=60.0, max_retries=10, max_tokens=200, azure_endpoint='https://nzcapstoneproject.openai.azure.com/', deployment_name='gpt-4o', openai_api_version='2024-06-01', openai_api_type='azure'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'azure_openai_text2speech', 'description': 'A wrapper around Azure OpenAI Text2Speech. Useful for when you need to convert text to speech. ', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string'}}, 'required': ['query']}}}]})

In [43]:
from langchain.agents import AgentType, AgentExecutor, create_structured_chat_agent
from langchain import hub

In [44]:
prompt = hub.pull("hwchase17/structured-chat-agent")

In [45]:
agent = create_structured_chat_agent(
    tools=[tts],
    llm=model,
    prompt=prompt
)

In [46]:
agent_executor = AgentExecutor(
    agent=agent, tools=[tts], verbose=True, handle_parsing_errors=True
)

In [48]:
agent_executor.invoke(
    {
        "input": "Can you tell me a joke and read it out load for me.",
    }
)



> Entering new AgentExecutor chain...
Thought: I will provide a brief history of Seattle and use the text-to-speech tool to read it out loud.

Action:
```
{
  "action": "azure_openai_text2speech",
  "action_input": "Seattle, located in the Pacific Northwest region of the United States, was originally inhabited by Native American tribes, particularly the Duwamish and Suquamish. The city was named after Chief Seattle, a leader of the Duwamish and Suquamish tribes. European settlers arrived in the mid-19th century, and the city was officially incorporated in 1869. Seattle's early economy was driven by logging, but it later became a hub for shipbuilding during World War I and II. The city experienced significant growth during the tech boom of the late 20th century, becoming home to major companies like Microsoft and Amazon. Today, Seattle is known for its vibrant cultural scene, coffee culture, and as a gateway to the natural beauty of the PacificC:\Users\spong\AppData\Local\Temp\tmpw70c

{'input': 'Can you tell me a brief history about Seattle, USA and read it out loud for me.',
 'output': "Seattle, located in the Pacific Northwest region of the United States, was originally inhabited by Native American tribes, particularly the Duwamish and Suquamish. The city was named after Chief Seattle, a leader of the Duwamish and Suquamish tribes. European settlers arrived in the mid-19th century, and the city was officially incorporated in 1869. Seattle's early economy was driven by logging, but it later became a hub for shipbuilding during World War I and II. The city experienced significant growth during the tech boom of the late 20th century, becoming home to major companies like Microsoft and Amazon. Today, Seattle is known for its vibrant cultural scene, coffee culture, and as a gateway to the natural beauty of the Pacific Northwest."}